In [15]:
from tracker import Tracker
from video import Video
from data import Data
from plotter import Plotter
from adapter import Adapter
from unit_converter import Unit_converter
import column_filter as cf
from datetime import datetime
from forces import Forces
import pandas as pd
import plotly.graph_objects as go
import re

In [2]:
reference_distance = 0.3
mass = 0.5
tracker = Tracker([6, 8, 10])
video = Video("videos/video0.mkv")
if not video.is_opened():
    print("Video not found")
    exit()

positions = []
objects = ["shoulder", "elbow", "wrist"]

In [9]:
for frame in video.get_frames():
    keypoints = tracker.get_keypoints(frame)

    if len(keypoints) > 0:
        positions.append(keypoints)


0: 384x640 1 person, 1100.2ms
Speed: 12.2ms preprocess, 1100.2ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 1050.1ms
Speed: 49.3ms preprocess, 1050.1ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 946.8ms
Speed: 15.6ms preprocess, 946.8ms inference, 8.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 887.0ms
Speed: 22.0ms preprocess, 887.0ms inference, 5.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 850.5ms
Speed: 15.6ms preprocess, 850.5ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 840.5ms
Speed: 15.7ms preprocess, 840.5ms inference, 12.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 847.0ms
Speed: 0.0ms preprocess, 847.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 833.1ms
Speed: 0.0ms preprocess, 833.1ms inference, 0.0ms postprocess p

In [10]:
adapter = Adapter(positions, objects, video.get_height())
data = Data(adapter.get_adapted_data(), objects)
df = data.get_data()
df[['rx_wrist', 'ry_wrist']]

,rx_wrist,ry_wrist
0,878.066667,363.822222
1,877.633333,363.338889
2,877.200000,362.855556
3,876.766667,362.372222
4,876.333333,361.888889
...,...,...
356,900.333333,338.333333
357,901.533333,337.800000
358,902.733333,337.266667
359,903.933333,336.733333


In [5]:
unit_converter = Unit_converter(df['r_wrist'].iloc[0], reference_distance, video.get_fps(), 1)
df = unit_converter.convert_position(df, cf.position_columns(df.columns))
df = unit_converter.convert_velocity(df, cf.velocity_columns(df.columns))
df = unit_converter.convert_acceleration(df, cf.acceleration_columns(df.columns))
df = unit_converter.convert_angular_velocity(df, ['angular_velocity'])
df = unit_converter.convert_time(df, ['time'])
plotter = Plotter(df)
df[['rx_wrist', 'ry_wrist']]

c:\Users\juanb\AppData\Local\Programs\Python\Python312\Lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




,rx_wrist,ry_wrist
0,1.849553,0.766352
1,1.848640,0.765334
2,1.847727,0.764316
3,1.846814,0.763298
4,1.845901,0.762280
...,...,...
356,1.896455,0.712663
357,1.898983,0.711539
358,1.901510,0.710416
359,1.904038,0.709292


In [11]:
plotter = Plotter(df)

In [11]:
df = Forces(df, mass).get_data_with_forces()
df[['rx_wrist', 'ry_wrist', 'fx_g', 'fy_g']]

,rx_wrist,ry_wrist,fx_g,fy_g
0,878.066667,363.822222,0,-4.905
1,877.633333,363.338889,0,-4.905
2,877.200000,362.855556,0,-4.905
3,876.766667,362.372222,0,-4.905
4,876.333333,361.888889,0,-4.905
...,...,...,...,...
356,900.333333,338.333333,0,-4.905
357,901.533333,337.800000,0,-4.905
358,902.733333,337.266667,0,-4.905
359,903.933333,336.733333,0,-4.905


In [17]:
df.to_csv(f"csv/panchos_data_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.csv", index = False)

In [7]:
df = pd.read_csv("csv/panchos_data_2024-04-22_21-23-17.csv")


In [12]:
reversed_unit_converter = unit_converter.get_reversed_unit_converter()
df = reversed_unit_converter.convert_position(df, cf.position_columns(df.columns))
df = reversed_unit_converter.convert_force(df, cf.force_columns(df.columns))
df[['rx_wrist', 'ry_wrist', 'fx_g', 'fy_g']]

,rx_wrist,ry_wrist,fx_g,fy_g
0,416858.162140,172722.947651,0.0,-2.59254
1,416652.439108,172493.487347,0.0,-2.59254
2,416446.716076,172264.027042,0.0,-2.59254
3,416240.993044,172034.566737,0.0,-2.59254
4,416035.270012,171805.106432,0.0,-2.59254
...,...,...,...,...
356,427429.161013,160622.213413,0.0,-2.59254
357,427998.855563,160369.015835,0.0,-2.59254
358,428568.550113,160115.818257,0.0,-2.59254
359,429138.244663,159862.620679,0.0,-2.59254


In [8]:
for column in [column for column in df.columns if re.match("ry_", column)]:
    df[column] = video.get_height() - df[column]
for column in [column for column in df.columns if re.match("fy_", column)]:
    df[column] = -1 * df[column]
df[['rx_wrist', 'ry_wrist', 'fx_g', 'fy_g']]

,rx_wrist,ry_wrist,fx_g,fy_g
0,416858.162140,-171642.947651,0.0,2.59254
1,416652.439108,-171413.487347,0.0,2.59254
2,416446.716076,-171184.027042,0.0,2.59254
3,416240.993044,-170954.566737,0.0,2.59254
4,416035.270012,-170725.106432,0.0,2.59254
...,...,...,...,...
356,427429.161013,-159542.213413,0.0,2.59254
357,427998.855563,-159289.015835,0.0,2.59254
358,428568.550113,-159035.818257,0.0,2.59254
359,429138.244663,-158782.620679,0.0,2.59254


In [9]:
vectors = [
    (df['rx_wrist'].values, df['ry_wrist'].values, df['fx_g'].values, df['fy_g'].values, 'g', (0, 255, 0)),
    (df['rx_wrist'].values, df['ry_wrist'].values, df['fx_not_g'].values, df['fy_not_g'].values, 'not_g', (255, 0, 0)),
]

In [13]:
video.show_with_vectors(vectors)

In [12]:
plotter.show_plot()

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

In [ ]:
video.close()

In [13]:
df

,rx_shoulder,ry_shoulder,rx_elbow,ry_elbow,rx_wrist,ry_wrist,vx_shoulder,vy_shoulder,vx_elbow,vy_elbow,...,r_wrist,theta_wrist,angular_velocity,time,fx_total,fy_total,fx_g,fy_g,fx_not_g,fy_not_g
0,448634.458153,-314710.129004,422544.557741,-239004.053244,416858.162140,-171642.947651,NaN,NaN,NaN,NaN,...,67614.975071,2.893083,NaN,0,NaN,NaN,0.0,2.59254,NaN,NaN
1,448634.458153,-314662.654458,422354.659558,-238893.279303,416652.439108,-171413.487347,NaN,NaN,NaN,NaN,...,67732.106442,2.891650,-0.001434,1,NaN,NaN,0.0,2.59254,NaN,NaN
2,448634.458153,-314615.179912,422164.761374,-238782.505363,416446.716076,-171184.027042,NaN,NaN,NaN,NaN,...,67849.237814,2.890216,-0.001434,2,NaN,NaN,0.0,2.59254,NaN,NaN
3,448634.458153,-314567.705366,421974.863191,-238671.731423,416240.993044,-170954.566737,NaN,NaN,NaN,NaN,...,67966.369185,2.888782,-0.001434,3,NaN,NaN,0.0,2.59254,NaN,NaN
4,448634.458153,-314520.230821,421784.965008,-238560.957482,416035.270012,-170725.106432,NaN,NaN,NaN,NaN,...,68083.500556,2.887348,-0.001434,4,NaN,NaN,0.0,2.59254,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356,445363.989440,-312621.248987,429117.144865,-226956.068500,427429.161013,-159542.213413,-0.002469,-0.076543,0.518519,-0.456790,...,67456.523759,2.978986,0.000112,356,0.040167,0.014863,0.0,2.59254,0.040167,-2.577677
357,445387.726713,-312565.862017,429433.641837,-226679.133650,427998.855563,-159289.015835,0.020679,-0.094136,0.581481,-0.513272,...,67423.595543,2.979118,0.000132,357,0.034849,0.012264,0.0,2.59254,0.034849,-2.580276
358,445411.463986,-312510.475047,429750.138809,-226402.198799,428568.550113,-159035.818257,0.043827,-0.111728,0.644444,-0.569753,...,67390.667328,2.979250,0.000132,358,0.029531,0.009665,0.0,2.59254,0.029531,-2.582875
359,445435.201258,-312455.088077,430066.635781,-226125.263948,429138.244663,-158782.620679,0.066975,-0.129321,0.707407,-0.626235,...,67357.739112,2.979382,0.000132,359,0.024213,0.007065,0.0,2.59254,0.024213,-2.585475


In [17]:
fig =go.Figure()
fig.add_traces(go.Scatter(x = df['time'], y = df['r_wrist'], mode="markers", name = 'rx_wrist'))

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'mode': 'markers',
              'name': 'rx_wrist',
              'type': 'scatter',
              'x': array([  0,   1,   2, ..., 358, 359, 360], dtype=int64),
              'y': array([      67615,       67732,       67849, ...,       67391,       67358,
                                67325])}],
    'layout': {'template': '...'}
})

In [20]:
df['r_wrist']

0      67614.975071
1      67732.106442
2      67849.237814
3      67966.369185
4      68083.500556
           ...     
356    67456.523759
357    67423.595543
358    67390.667328
359    67357.739112
360    67324.810896
Name: r_wrist, Length: 361, dtype: float64

In [23]:
df.columns

Index(['rx_shoulder', 'ry_shoulder', 'rx_elbow', 'ry_elbow', 'rx_wrist',
       'ry_wrist', 'vx_shoulder', 'vy_shoulder', 'vx_elbow', 'vy_elbow',
       'vx_wrist', 'vy_wrist', 'ax_shoulder', 'ay_shoulder', 'ax_elbow',
       'ay_elbow', 'ax_wrist', 'ay_wrist', 'r_wrist', 'theta_wrist',
       'angular_velocity', 'time', 'fx_total', 'fy_total', 'fx_g', 'fy_g',
       'fx_not_g', 'fy_not_g'],
      dtype='object')